# Lesson 2: Explore Carbon Intensity on the Grid

* In this classroom, the libraries have been already installed for you.
* If you would like to run this code on your own machine, make sure to get your Electricity Maps API from their [website](https://www.electricitymaps.com/free-tier-api).

#### Load the Electricity Maps API for this notebook

In [31]:
from helper import load_env
load_env()

### Electricity Map's live carbon intensity  endpoint

* Set location coordinates.

Remember, you can get the coordinates from [Google Maps](https://www.google.com/maps).

In [32]:
# Create a dictionary with the coordinates
coordinates = {
    "lat":34.00906474557528,
    "lon": -118.4984580927553
}

* Request from the endpoint.

In [33]:
# Build the url
url= f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"

# Print the endpoint
print("Endpoint: " + str(url))

Endpoint: https://api.electricitymap.org/v3/carbon-intensity/latest?lat=34.00906474557528&lon=-118.4984580927553


* Prepare the get request.

In [34]:
import requests
import helper

In [35]:
request = requests.get(
    url,
    headers={"auth-token": helper.load_emaps_api_key()})

In [36]:
# This byte format is more compact
request.content
type(request.content)

bytes

> Note: This byte format is more compact and often not human-readable. Additionally, it is not possible to access the fields within this byte data as we would access with key-value pairs in a dictionary. This is why we use json.loads()

In [37]:
import json

In [38]:
json.loads(request.content)

{'zone': 'US-CAL-LDWP',
 'carbonIntensity': 402,
 'datetime': '2024-10-04T19:00:00.000Z',
 'updatedAt': '2024-10-04T18:46:42.545Z',
 'createdAt': '2024-10-01T19:51:44.203Z',
 'emissionFactorType': 'lifecycle',
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

* Use the live power breakdown endpoint.

In [39]:
# Build the url
url = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"

In [40]:
print(url)

https://api.electricitymap.org/v3/power-breakdown/latest?lat=34.00906474557528&lon=-118.4984580927553


In [41]:
request = requests.get(
    url,
    headers={"auth-token": helper.load_emaps_api_key()})

In [42]:
power_breakdown = json.loads(request.content)

# Print the content
power_breakdown

{'zone': 'US-CAL-LDWP',
 'datetime': '2024-10-04T19:00:00.000Z',
 'updatedAt': '2024-10-04T18:46:42.545Z',
 'createdAt': '2024-10-01T19:51:44.203Z',
 'powerConsumptionBreakdown': {'nuclear': 0,
  'geothermal': 0,
  'biomass': 0,
  'coal': 718,
  'wind': 48,
  'solar': 1445,
  'hydro': 0,
  'gas': 956,
  'oil': 0,
  'unknown': 123,
  'hydro discharge': 0,
  'battery discharge': 0},
 'powerProductionBreakdown': {'nuclear': None,
  'geothermal': None,
  'biomass': None,
  'coal': 718,
  'wind': 48,
  'solar': 1445,
  'hydro': 0,
  'gas': 956,
  'oil': None,
  'unknown': 123,
  'hydro discharge': None,
  'battery discharge': None},
 'powerImportBreakdown': {},
 'powerExportBreakdown': {},
 'fossilFreePercentage': 45,
 'renewablePercentage': 45,
 'powerConsumptionTotal': 3291,
 'powerProductionTotal': 3291,
 'powerImportTotal': None,
 'powerExportTotal': None,
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

* Print some specific values.

In [43]:
power_breakdown['renewablePercentage']

45

In [44]:
power_breakdown['fossilFreePercentage']

45

In [45]:
# Power Consumption Breakdown in MegaWatts
power_breakdown['powerConsumptionBreakdown']

{'nuclear': 0,
 'geothermal': 0,
 'biomass': 0,
 'coal': 718,
 'wind': 48,
 'solar': 1445,
 'hydro': 0,
 'gas': 956,
 'oil': 0,
 'unknown': 123,
 'hydro discharge': 0,
 'battery discharge': 0}

* Do some math to understand better the values above.

In [46]:
import numpy as np

In [47]:
total_consumption = power_breakdown['powerConsumptionTotal']

In [22]:
total_consumption

3291

In [48]:
consumption_percent = {
    k: np.round((v/total_consumption) * 100)
    for k,v
    in power_breakdown['powerConsumptionBreakdown'].items()}
consumption_percent

{'nuclear': np.float64(0.0),
 'geothermal': np.float64(0.0),
 'biomass': np.float64(0.0),
 'coal': np.float64(22.0),
 'wind': np.float64(1.0),
 'solar': np.float64(44.0),
 'hydro': np.float64(0.0),
 'gas': np.float64(29.0),
 'oil': np.float64(0.0),
 'unknown': np.float64(4.0),
 'hydro discharge': np.float64(0.0),
 'battery discharge': np.float64(0.0)}

#### Helper function for the power_stats

In [49]:
import helper, requests, json, numpy as np
def power_stats(lat,lon, api_key=helper.load_emaps_api_key()):
    coordinates = { "lat": lat, "lon": lon }

    url_intensity = f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_intensity = requests.get(url_intensity, headers={"auth-token": api_key})
    intensity = json.loads(request_intensity.content)

    url_breakdown = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_breakdown = requests.get(url_breakdown, headers={"auth-token": api_key})
    breakdown = json.loads(request_breakdown.content)

    breakdown_abridged = {
        'renewablePercentage': breakdown['renewablePercentage'],
        'fossilFreePercentage': breakdown['fossilFreePercentage'],
        'powerConsumptionBreakdown': breakdown['powerConsumptionBreakdown'],
        'consumption_percent': {
            k: np.round((v/breakdown['powerConsumptionTotal']) * 100) 
            for k, v 
            in breakdown['powerConsumptionBreakdown'].items()
        },
    }
    
    return intensity, breakdown_abridged

In [50]:
# Coordinates from a landmark in Taiwan, shown by the instructor in the explanation
intensity, breakdown = power_stats(
    lat=25.0356575108668,
    lon=121.52010809479746)

In [51]:
# Coordinates from a landmark in India
intensity, breakdown = power_stats(
    lat=11.127123,
    lon=78.656891)

In [52]:
intensity

{'zone': 'IN-SO',
 'carbonIntensity': 618,
 'datetime': '2024-10-04T19:00:00.000Z',
 'updatedAt': '2024-10-04T18:46:46.977Z',
 'createdAt': '2024-10-01T19:51:44.203Z',
 'emissionFactorType': 'lifecycle',
 'isEstimated': True,
 'estimationMethod': 'MODE_BREAKDOWN'}

In [53]:
breakdown

{'renewablePercentage': 20,
 'fossilFreePercentage': 25,
 'powerConsumptionBreakdown': {'nuclear': 2113,
  'geothermal': 0,
  'biomass': 0,
  'coal': 29513,
  'wind': 1836,
  'solar': 0,
  'hydro': 5960,
  'gas': 209,
  'oil': 8,
  'unknown': 0,
  'hydro discharge': 0,
  'battery discharge': 0},
 'consumption_percent': {'nuclear': np.float64(5.0),
  'geothermal': np.float64(0.0),
  'biomass': np.float64(0.0),
  'coal': np.float64(74.0),
  'wind': np.float64(5.0),
  'solar': np.float64(0.0),
  'hydro': np.float64(15.0),
  'gas': np.float64(1.0),
  'oil': np.float64(0.0),
  'unknown': np.float64(0.0),
  'hydro discharge': np.float64(0.0),
  'battery discharge': np.float64(0.0)}}

### Do it yourself! 
* Get coordinates from a location you want to retrieve the information we got before!